# Quantum Wavepacket Barrier Tunneling Simulation

Interactive 3D visualization of quantum wavepacket dynamics through potential barriers.

**Instructions:** Run all cells, then use the interactive controls to adjust barrier height, wave momentum, and time step.

**Note:** This is a simplified version optimized for web viewing. For the full interactive version with all controls, see the [GitHub repository](https://github.com/Vis-42/_space/tree/main/py/qm_sim).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import hbar, m_e
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
from ipywidgets import interact, FloatSlider, IntSlider

# Simplified parameters for web performance
GRID_SIZE = 1000  # Reduced for faster computation
DOMAIN_LENGTH = 2e-8
CFL_FACTOR = 0.4
BOUNDARY_WIDTH_FRACTION = 0.05
BARRIER_WIDTH_FRACTION = 0.2

class QuantumPhysics:
    def __init__(self, N, L, dt):
        self.N = N
        self.L = L
        self.dt = dt
        self.dx = L / N
        self.x = np.linspace(0, L, N)
        self.A = None
        self.B = None
        self.operators_valid = False

    def create_gaussian_wave(self, k0, sigma, x_center):
        psi = np.exp(-((self.x - x_center) ** 2) / (2 * sigma ** 2), dtype=complex)
        psi *= np.exp(1j * k0 * self.x)
        return psi / np.linalg.norm(psi)

    def create_barrier_potential(self, amplitude):
        V = np.zeros(self.N)
        barrier_width = int(BARRIER_WIDTH_FRACTION * self.N)
        center = self.N // 2
        start = center - barrier_width // 2
        end = center + barrier_width // 2
        V[start:end] = amplitude
        return V

    def update_evolution_operators(self, V):
        if self.operators_valid:
            return
        a_diag = (1 + 1j * hbar * self.dt / (2 * m_e * self.dx**2) +
                 1j * self.dt / (2 * hbar) * V)
        b_diag = (1 - 1j * hbar * self.dt / (2 * m_e * self.dx**2) -
                 1j * self.dt / (2 * hbar) * V)
        off_diag = -1j * hbar * self.dt / (4 * m_e * self.dx**2) * np.ones(self.N - 1)
        self.A = diags([off_diag, a_diag, off_diag], [-1, 0, 1], format='csr')
        self.B = diags([-off_diag, b_diag, -off_diag], [-1, 0, 1], format='csr')
        self.operators_valid = True

    def apply_absorbing_boundaries(self, psi):
        boundary_width = int(BOUNDARY_WIDTH_FRACTION * self.N)
        for i in range(boundary_width):
            damping = np.exp(-((boundary_width - i) / boundary_width)**2)
            psi[i] *= damping
        for i in range(self.N - boundary_width, self.N):
            damping = np.exp(-((i - (self.N - boundary_width)) / boundary_width)**2)
            psi[i] *= damping
        return psi

    def evolve_wavefunction(self, psi, V, steps):
        self.update_evolution_operators(V)
        for _ in range(abs(steps)):
            if steps > 0:
                psi = spsolve(self.A, self.B @ psi)
            else:
                psi = spsolve(self.B, self.A @ psi)
            psi = self.apply_absorbing_boundaries(psi)
        return psi

    def invalidate_operators(self):
        self.operators_valid = False

# Initialize physics engine
dt = CFL_FACTOR * m_e * (DOMAIN_LENGTH / GRID_SIZE)**2 / hbar
physics = QuantumPhysics(GRID_SIZE, DOMAIN_LENGTH, dt)

print("Quantum physics engine initialized!")


In [ ]:
def plot_wavepacket(barrier_height=1e-18, momentum=6e10, time_steps=0):
    """Interactive plot of quantum wavepacket evolution."""
    # Create initial wavepacket
    k0 = momentum
    sigma = 2e-10
    x_wave0 = DOMAIN_LENGTH / 4
    psi = physics.create_gaussian_wave(k0, sigma, x_wave0)
    
    # Create potential barrier
    V = physics.create_barrier_potential(barrier_height)
    physics.invalidate_operators()
    
    # Evolve wavefunction
    if time_steps > 0:
        psi = physics.evolve_wavefunction(psi, V, time_steps)
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    
    # Probability density
    prob_density = np.abs(psi)**2
    ax1.plot(physics.x * 1e9, prob_density, 'b-', linewidth=2, label=r'$|\psi|^2$')
    ax1.fill_between(physics.x * 1e9, 0, prob_density, alpha=0.3)
    
    # Potential barrier
    barrier_center = DOMAIN_LENGTH / 2
    barrier_start = barrier_center - BARRIER_WIDTH_FRACTION * DOMAIN_LENGTH / 2
    barrier_end = barrier_center + BARRIER_WIDTH_FRACTION * DOMAIN_LENGTH / 2
    ax1.axvspan(barrier_start * 1e9, barrier_end * 1e9, alpha=0.3, color='orange', label='Potential Barrier')
    
    ax1.set_xlabel('Position (nm)')
    ax1.set_ylabel('Probability Density')
    ax1.set_title(f'Quantum Wavepacket Evolution (Time Step: {time_steps})')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Real and imaginary components
    ax2.plot(physics.x * 1e9, np.real(psi), 'r-', linewidth=1.5, label='Re(ψ)', alpha=0.7)
    ax2.plot(physics.x * 1e9, np.imag(psi), 'g-', linewidth=1.5, label='Im(ψ)', alpha=0.7)
    ax2.axvspan(barrier_start * 1e9, barrier_end * 1e9, alpha=0.2, color='orange')
    ax2.set_xlabel('Position (nm)')
    ax2.set_ylabel('Wavefunction Amplitude')
    ax2.set_title('Real and Imaginary Components')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    prob_density = np.abs(psi)**2
    center_of_mass = np.sum(physics.x * prob_density) / np.sum(prob_density)
    print(f"Center of mass: {center_of_mass * 1e9:.2f} nm")
    print(f"Barrier height: {barrier_height:.2e} J")
    print(f"Wave momentum: {momentum:.2e} m⁻¹")

# Create interactive widget
interact(plot_wavepacket,
         barrier_height=FloatSlider(min=0, max=2e-17, step=1e-18, value=1e-18, description='Barrier Height (J)'),
         momentum=FloatSlider(min=1e10, max=1e11, step=1e9, value=6e10, description='Momentum (m⁻¹)'),
         time_steps=IntSlider(min=0, max=5000, step=50, value=0, description='Time Steps'))
